In [1]:
import pandas as pd
import re
from difflib import get_close_matches

NORMALISATION DES TRAITEMENTS VIH

In [4]:
# ------------------------------------------------------------------
# 1. Charger les fichiers
# ------------------------------------------------------------------
df_ref = pd.read_excel("data/med hepatite coriger.xlsx")
df_arv = pd.read_excel("data/Med_Hep.xlsx")

print(f"→ {len(df_ref)} médicaments dans le dictionnaire VIH")
print(f"→ {len(df_arv)} lignes ARV à traiter")


→ 79 médicaments dans le dictionnaire VIH
→ 286 lignes ARV à traiter


In [6]:
# ------------------------------------------------------------------
# 2. Préparation du dictionnaire VIH
# ------------------------------------------------------------------
df_ref["Med"] = df_ref["Med"].fillna("").astype(str)
df_ref["DCI"] = df_ref["DCI"].fillna("").astype(str)


In [8]:
# Liste des médicaments (nom commercial + nom de base + dci)
medicaments = []

for _, row in df_ref.iterrows():
    nom = row["Med"].strip()
    dci = row["DCI"].strip()

    if nom and dci:
        nom_base = re.split(r'\s+\d|,|\s+\(', nom)[0].strip().upper()
        medicaments.append({
            "nom_complet": nom,
            "nom_base": nom_base,
            "nom_base_lower": nom_base.lower(),
            "dci": dci
        })

In [9]:
# Set des DCI pour recherche rapide
dci_set = set()
for d in df_ref["DCI"]:
    if d:
        dci_set.add(d.lower())
        for comp in re.split(r"[\+/]", d):
            comp = comp.strip()
            if comp:
                dci_set.add(comp.lower())

print(f"✓ Dictionnaire chargé : {len(medicaments)} médicaments")

✓ Dictionnaire chargé : 79 médicaments


In [10]:
# ------------------------------------------------------------------
# 3. Fonctions utilitaires simples
# ------------------------------------------------------------------

def extraire_nom_commercial(txt):
    m = re.search(r"\(([^)]+)\)", txt)
    return m.group(1).strip() if m else None

def enlever_parentheses(txt):
    return re.sub(r"\s*\([^)]*\)", "", txt).strip()

def chercher_dci(med_str):
    if not med_str or not med_str.strip():
        return "VIDE"

    med_str = med_str.strip()

    # 1. regarder parenthèses → nom commercial
    nom_com = extraire_nom_commercial(med_str)
    if nom_com:
        cle = nom_com.strip().upper()
    else:
        cle = enlever_parentheses(med_str).upper()

    cle_lower = cle.lower()

    # 2. correspondance exacte sur nom de base
    for med in medicaments:
        if med["nom_base"] == cle:
            return med["dci"]

    # 3. correspondance partielle
    for med in medicaments:
        if med["nom_base"].startswith(cle) or cle.startswith(med["nom_base"]):
            return med["dci"]

    # 4. recherche dans nom complet
    for med in medicaments:
        if cle in med["nom_complet"].upper():
            return med["dci"]

    # 5. matching approx
    bases = [m["nom_base_lower"] for m in medicaments]
    match = get_close_matches(cle_lower, bases, n=1, cutoff=0.8)
    if match:
        for m in medicaments:
            if m["nom_base_lower"] == match[0]:
                return m["dci"]

    # 6. déjà une DCI ?
    if cle_lower in dci_set:
        for m in medicaments:
            if cle_lower in m["dci"].lower():
                return m["dci"]

    # # 7 matching approx dans DCI
    # match = get_close_matches(cle_lower, list(dci_set), n=1, cutoff=0.9)
    # if match:
    #     for m in medicaments:
    #         if match[0] in m["dci"].lower():
    #             return m["dci"]

    return f"INTROUVABLE ({cle})"


In [11]:
# ------------------------------------------------------------------
# 4. Normalisation du traitement ARV
# ------------------------------------------------------------------

def normaliser_traitement(traitement):
    if pd.isna(traitement) or not str(traitement).strip():
        return "VIDE"

    items = [t.strip() for t in str(traitement).split("+")]

    dcis = []
    for item in items:
        if item:
            d = chercher_dci(item)
            d = d.replace(" + ", " / ")
            dcis.append(d)

    return " + ".join(dcis)

In [12]:
# ------------------------------------------------------------------
# 5. Appliquer sur tout le DataFrame
# ------------------------------------------------------------------

print("\nTraitement en cours...")
df_arv["DCI"] = df_arv["Med_Hep"].apply(normaliser_traitement)

nb_introuv = df_arv["DCI"].str.contains("INTROUVABLE", na=False).sum()
nb_vides = (df_arv["DCI"] == "VIDE").sum()

print(f"✓ Normalisation terminée")
print(f"  → Trouvés : {len(df_arv) - nb_introuv - nb_vides}")
print(f"  → Introuvables : {nb_introuv}")
print(f"  → Vides : {nb_vides}")



Traitement en cours...
✓ Normalisation terminée
  → Trouvés : 249
  → Introuvables : 37
  → Vides : 0


In [13]:
# ------------------------------------------------------------------
# 6. Sauvegarde
# ------------------------------------------------------------------
df_arv.to_excel("data/out/Med_Hep_dci.xlsx", index=False)
print("\nFichier créé : data/out/Med_Hep_dci.xlsx")
print("="*60)



Fichier créé : data/out/Med_Hep_dci.xlsx


# ATC

In [11]:
print("Lecture des fichiers...")
df_arv = pd.read_excel("data/out/Med_Hep_dci.xlsx")
df_vih = pd.read_excel("data/bdd.xlsx")

print(f"   → {len(df_arv)} lignes ARV")
print(f"   → {len(df_vih)} lignes VIH")

Lecture des fichiers...
   → 286 lignes ARV
   → 15965 lignes VIH


In [12]:
# ------------------------------------------------------------
# 1) NORMALISER LES DCI
# ------------------------------------------------------------
def norm(x):
    return " ".join(str(x).lower().strip().split())


In [13]:
# ------------------------------------------------------------
# 2) CRÉATION DU MAPPING DCI → ATC
# ------------------------------------------------------------

mapping = {}

for _, row in df_vih.iterrows():
    dci = str(row["dci"])
    atc = str(row["atc"])

    # DCI complète
    dci_norm = norm(dci)
    mapping[dci_norm] = atc

    # Si combinaison (lamivudine + zidovudine)
    if "+" in dci:
        parts = [norm(p) for p in dci.split("+")]
        for p in parts:
            if p not in mapping:
                mapping[p] = atc

    # Si "/" dans le fichier VIH
    if "/" in dci:
        dci_slash = norm(dci.replace("/", "+"))
        mapping[dci_slash] = atc

print(f" Mapping créé : {len(mapping)} clés DCI")


 Mapping créé : 4098 clés DCI


In [14]:
# ------------------------------------------------------------
# 3) GÉNÉRATION DE LA COLONNE ATC
# ------------------------------------------------------------

ATC_output = []

for _, row in df_arv.iterrows():
    dci_full = str(row["DCI"])

    # Séparation des médicaments d’un traitement
    meds = [m.strip() for m in dci_full.split("+")]

    atc_codes = []

    for med in meds:

        med_norm = norm(med)

        # 1) recherche directe
        if med_norm in mapping:
            atc_codes.append(mapping[med_norm])
            continue

        # 2) si "/" dans la DCI normalisée
        if "/" in med:
            med_plus = med_norm.replace("/", "+")
            if med_plus in mapping:
                atc_codes.append(mapping[med_plus])
                continue

        # 3) essayer permutations pour combinaisons
        if "/" in med or "+" in med:

            # extraire composants
            parts = re.split(r"[+/]", med)
            parts = [norm(p) for p in parts]

            combo1 = " + ".join(parts)
            combo2 = " + ".join(parts[::-1])

            if combo1 in mapping:
                atc_codes.append(mapping[combo1])
                continue
            if combo2 in mapping:
                atc_codes.append(mapping[combo2])
                continue

        # 4) rien trouvé
        # print(f"⚠️ ATC non trouvé pour: {med}")
        atc_codes.append("NON_TROUVE")

    ATC_output.append(" + ".join(atc_codes))



In [15]:

# ------------------------------------------------------------
# 4) AJOUTER LA COLONNE ET SAUVEGARDER
# ------------------------------------------------------------

df_arv["ATC"] = ATC_output

out = "data/out/Med_Hep_dci_atc.xlsx"
df_arv.to_excel(out, index=False)

print("\n Fichier généré :", out)
print("\n APERÇU :")
print(df_arv[["Med_Hep", "DCI", "ATC"]].head(10).to_string())

# Statistique
nb_missing = df_arv["ATC"].str.contains("NON_TROUVE").sum()
print("\n ATC NON TROUVÉS :", nb_missing)



 Fichier généré : data/out/Med_Hep_dci_atc.xlsx

 APERÇU :
                                    Med_Hep                                    DCI                   ATC
0                         Copegus + Pegasys     Ribavirine + PEGINTERFÉRON ALFA-2A     J05AP01 + L03AB11
1                   Rébétol + Viraferon Peg     Ribavirine + Péginterféron alfa-2b     J05AP01 + L03AB10
2                             Viraferon Peg                  Péginterféron alfa-2b               L03AB10
3                                   Harvoni                 LÉDIPASVIR; SOFOSBUVIR               J05AP51
4                                   Epclusa                SOFOSBUVIR; VELPATASVIR               J05AP55
5                                 Baraclude                  ENTÉCAVIR MONOHYDRATÉ               J05AF10
6                                 Viraféron                     Interféron alfa-2b               L03AB05
7                      Interferon + Rébétol  INTROUVABLE (INTERFERON) + Ribavirine  NON_TROUVE + J05

In [16]:
import pandas as pd
import numpy as np

# Charger le fichier
df = pd.read_excel("data/out/Med_Hep_dci_atc.xlsx")

# Remplacer NaN par des chaînes vides pour éviter l'erreur split
df[["Med_Hep", "DCI", "ATC"]] = df[["Med_Hep", "DCI", "ATC"]].fillna("")

# Trouver le nombre max de médicaments dans tout le fichier
df["n_meds"] = df["Med_Hep"].apply(lambda x: len([m.strip() for m in str(x).split("+")]))
max_meds = df["n_meds"].max()

# Fonction de transformation
def process_row(row):
    arv_list = [x.strip() for x in str(row["Med_Hep"]).split("+") if x.strip() != ""]
    dci_list = [x.strip() for x in str(row["DCI"]).split("+") if x.strip() != ""]
    atc_list = [x.strip() for x in str(row["ATC"]).split("+") if x.strip() != ""]

    data = {}

    for i in range(max_meds):
        data[f"Med_{i+1}"]      = arv_list[i] if i < len(arv_list) else np.nan
        data[f"DCI_Med{i+1}"]   = dci_list[i] if i < len(dci_list) else np.nan
        data[f"ATC_Med{i+1}"]   = atc_list[i] if i < len(atc_list) else np.nan

    return pd.Series(data)

# Appliquer à toutes les lignes
new_cols = df.apply(process_row, axis=1)

# Fusionner
df_final = pd.concat([df, new_cols], axis=1)

df_final.head()


,Med_Hep,DCI,ATC,n_meds,Med_1,DCI_Med1,ATC_Med1,Med_2,DCI_Med2,ATC_Med2,Med_3,DCI_Med3,ATC_Med3,Med_4,DCI_Med4,ATC_Med4,Med_5,DCI_Med5,ATC_Med5
0,Copegus + Pegasys,Ribavirine + PEGINTERFÉRON ALFA-2A,J05AP01 + L03AB11,2,Copegus,Ribavirine,J05AP01,Pegasys,PEGINTERFÉRON ALFA-2A,L03AB11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rébétol + Viraferon Peg,Ribavirine + Péginterféron alfa-2b,J05AP01 + L03AB10,2,Rébétol,Ribavirine,J05AP01,Viraferon Peg,Péginterféron alfa-2b,L03AB10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Viraferon Peg,Péginterféron alfa-2b,L03AB10,1,Viraferon Peg,Péginterféron alfa-2b,L03AB10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Harvoni,LÉDIPASVIR; SOFOSBUVIR,J05AP51,1,Harvoni,LÉDIPASVIR; SOFOSBUVIR,J05AP51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Epclusa,SOFOSBUVIR; VELPATASVIR,J05AP55,1,Epclusa,SOFOSBUVIR; VELPATASVIR,J05AP55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Sauvegarder
df_final.to_excel("data/out/Med_Hep_dci_atc_split.xlsx", index=False)